In [13]:
%matplotlib inline

from astropy.io import fits
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import random
from scipy import ndimage

ppp = 0.2 # parsec per pixel
pc_to_cm = 9.521*1e36 # parsec^2 to cm^2

plt.style.use({"image.origin": "lower", "image.interpolation": "nearest"})

FITS_PATH = '../data/fits/PROMISE-Q1-8micron-filled-v0_3.fits'
# FITS_PATH = '../cnn/fits_for_annotation/box(50316.355, 6033.3671, 234.3168, 178.3296, 0.0).fits'
# Open img using memmap to not load the whole image into memory
imageHDU = fits.open(FITS_PATH, memmap=True, mode='denywrite')

# read catalog.csv
catalog = pd.read_csv('../data/catalog_v2.2.csv', index_col=False, dtype={"x_center": np.int32, "y_center": np.int32, "approx_size": np.int32, "box_x1": np.int32, "box_x2": np.int32, "box_y1": np.int32, "box_y2": np.int32, "mask_file": str})

# catalog bigger then 1000px²
# catalog = catalog[catalog["approx_size"] >= 10000]
# catalog = catalog.reset_index(drop=True)

# plot cloud nr 4357
index = 1824 #2377 random.randint(0, len(catalog)-1)#7910 #random.randint(0, len(catalog)-1) #4357 
padding = 80
row = catalog.iloc[index]



extent = (-padding-0.5, row["box_x2"]+padding-row["box_x1"]-0.5, -padding-0.5, row["box_y2"]+padding-row["box_y1"]-0.5)

rand_mask = np.load("../data/masks/"+row["mask_file"])

data = np.array(imageHDU[0].data[row["box_y1"]-padding:row["box_y2"]+padding, row["box_x1"]-padding:row["box_x2"]+padding], dtype=np.float32) # type: ignore

nearby = catalog[(catalog["box_x2"] > row["x_center"]-padding) & (catalog["box_x1"] < row["x_center"]+padding) & (catalog["box_y2"] > row["y_center"]-padding) & (catalog["box_y1"] < row["y_center"]+padding)]
nearby = nearby[nearby["x_center"] != row["x_center"]] # remove self

print(nearby.head())

for index, nrow in nearby.iterrows():
    mask = np.load("../data/masks/"+nrow["mask_file"])

    if mask.shape != (nrow["box_y2"]-nrow["box_y1"], nrow["box_x2"]-nrow["box_x1"]):
        print("something is weird with the box", index)
        mask.resize((nrow["box_y2"]-nrow["box_y1"], nrow["box_x2"]-nrow["box_x1"]))

    nr, nc = mask.shape
    for r in range(nr):
        for c in range(nc):
            if mask[r,c] == True:
                data[r,c] = np.nan

    #mask = np.where(mask == True, np.nan, 1000)
    print(mask.shape)
    print((nrow["box_y2"]-nrow["box_y1"], nrow["box_x2"]-nrow["box_x1"]))

    data[(nrow["box_y1"]-row["box_y1"]+padding):(nrow["box_y2"]-row["box_y1"]+padding), (nrow["box_x1"]-row["box_x1"]+padding):(nrow["box_x2"]-row["box_x1"]+padding)] = data[(nrow["box_y1"]-row["box_y1"]+padding):(nrow["box_y2"]-row["box_y1"]+padding), (nrow["box_x1"]-row["box_x1"]+padding):(nrow["box_x2"]-row["box_x1"]+padding)] + mask



data_box_Ioff = np.array(imageHDU[0].data[row["box_y1"]:row["box_y2"], row["box_x1"]: row["box_x2"]], dtype=np.float32) # type: ignore
data_box_Iobs = np.array(imageHDU[0].data[row["box_y1"]:row["box_y2"], row["box_x1"]: row["box_x2"]], dtype=np.float32) # type: ignore

rows, cols = rand_mask.shape
for r in range(rows):
    for c in range(cols):
        if rand_mask[r,c] == True:
            data_box_Ioff[r,c] = None
        else:
            data_box_Iobs[r,c] = None

flat_data_box_Iobs = np.sort(data_box_Iobs.flatten())
I_fg = np.nanmean(flat_data_box_Iobs[:10])
I_obs = np.nanmean(flat_data_box_Iobs)
I_off = np.nanmean(data_box_Ioff.flatten())

tau8 = -np.log((I_obs - I_fg)/(I_off - I_fg))
kappa8 = 7.5 #cm^2 /g
sigma8 = tau8/kappa8
approx_size_of_cloud = row[2]


fat_sun = approx_size_of_cloud*ppp**2*pc_to_cm*sigma8/1000


print(f'approximate size of cloud: {approx_size_of_cloud}')
print(f'tau_8: {tau8} sigma_8: {sigma8}')
print(f'approximate mass of cloud: {fat_sun} kg or {fat_sun/1.9885e30} solar masses')


#---------------------------------------------------- observer pixel corresponds to an angle IN HEADER: [CD2_2], distance d=3,5 kpc------------100s solar masses smallers clouds???--------biggest 10^5 solar masses-------------------------------------------------------------------------
# for smaller clouds the fg is harder to find. We can't just pick the darkest pixel. Do some saturated based???
# Cosmic conspiracy.

cmap1 = ListedColormap(['none', 'white']) # type: ignore

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 2, 1)

ax.imshow(data, cmap='inferno', extent=extent, norm=ImageNormalize(stretch=SqrtStretch())) #type: ignore
ax.imshow(rand_mask, cmap=cmap1, alpha=0.2)
ax.plot(row["x_center"]-row["box_x1"], row["y_center"]-row["box_y1"], 'rx', alpha=0.3)
ax.title.set_text("Size: " + str(int(row["approx_size"])) + "px² | Object: " + str(index+1) + "/" + str(len(catalog))+"\n"+"X: " + str(row["x_center"]) + " | Y: " + str(row["y_center"]))
ax.plot([-0.5, row["box_x2"]-row["box_x1"]-0.5, row["box_x2"]-row["box_x1"]-0.5, -0.5, -0.5], [-0.5, -0.5, row["box_y2"]-row["box_y1"]-0.5, row["box_y2"]-row["box_y1"]-0.5, -0.5], 'r-', alpha=0.3)



ax = fig.add_subplot(1, 2, 2)
ax.imshow(data, cmap='inferno', extent=extent, norm=ImageNormalize(stretch=SqrtStretch()))
ax.title.set_text("Comparison")

      x_center  y_center  approx_size  box_x1  box_x2  box_y1  box_y2  \
1836     24574      5960         2660   24543   24613    5919    6008   
1839     24462      5948         1467   24439   24485    5925    5982   
1850     24424      5951           76   24419   24430    5947    5957   

           mask_file  
1836  24574_5960.npy  
1839  24462_5948.npy  
1850  24424_5951.npy  
(89, 70)
(89, 70)


ValueError: operands could not be broadcast together with shapes (58,6) (89,70) 